# ADS Assignment 
# Akanksha Telagam Setty

In [43]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random

# Dataset 

In [44]:
df= pd.read_csv("diabetes2.csv")
df.head

<bound method NDFrame.head of      Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0              6      148             72             35        0  33.6   
1              1       85             66             29        0  26.6   
2              8      183             64              0        0  23.3   
3              1       89             66             23       94  28.1   
4              0      137             40             35      168  43.1   
..           ...      ...            ...            ...      ...   ...   
763           10      101             76             48      180  32.9   
764            2      122             70             27        0  36.8   
765            5      121             72             23      112  26.2   
766            1      126             60              0        0  30.1   
767            1       93             70             31        0  30.4   

     DiabetesPedigreeFunction  Age  Outcome  
0                       0.627   50 

In [45]:
#Features
x = df[['Glucose', 'BloodPressure', 'Insulin', 'BMI', 'Age', 'DiabetesPedigreeFunction']]
y = df['Outcome']

## Train-test split

In [46]:
#Train test split
x_train = x.iloc[:700,:]
x_test = x.iloc[700:,:]


'''y_list = df['Outcome'].tolist()
y_train = y_list[:700]
y_test = y_list[700:]
'''
y_train = y.iloc[:700]
y_test = y.iloc[700:]

## Sigmoid Activation

In [47]:
def sigmoid(input):    
    output = 1 / (1 + np.exp(-input))
    return output

## ReLU Activation

In [48]:
def ReLU(input):
  data = [max(0,value) for value in input]
  return np.array(data, dtype=float) 

## Optimization

In [49]:
#optimization function
def optimize(x, y,learning_rate,iterations,parameters, activation): 
    size = x.shape[0]
    weight = parameters["weight"] 
    bias = parameters["bias"]
    for i in range(iterations): 
      if activation == "sigmoid":
        sigma = sigmoid(np.dot(x, weight) + bias)
      elif activation == "relu":
        sigma = ReLU(np.dot(x, weight) + bias)
      loss = -1/size * np.sum(y * np.log(sigma)) + (1 - y) * np.log(1-sigma)
      dW = 1/size * np.dot(x.T, (sigma - y))
      db = 1/size * np.sum(sigma - y)
      weight -= learning_rate * dW
      bias -= learning_rate * db 
    
    parameters["weight"] = weight
    parameters["bias"] = bias
    parameters["activation"] = activation
    return parameters

In [50]:
init_parameters = {} 
init_parameters["weight"] = np.random.normal( size=x.shape[1]) #np.zeros(x.shape[1])
init_parameters["bias"] = 0

In [51]:
def train(x, y, learning_rate,iterations,activation):
    parameters_out = optimize(x, y, learning_rate, iterations ,init_parameters, activation)
    return parameters_out

In [52]:
parameters_out = train(x_train, y_train, learning_rate = 0.001, iterations = 200, activation = 'sigmoid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in log
  # This is added back by InteractiveShellApp.init_path()


In [53]:
parameters_out

{'activation': 'sigmoid',
 'bias': -0.016679426290196614,
 'weight': array([ 0.02094779, -0.11745883,  0.01734338,  0.0081973 ,  0.19856327,
        -0.42335198])}

### Testing 


In [54]:
#Predictions
output_values = np.dot(x_test, parameters_out['weight'])+parameters_out['bias']
if parameters_out['activation'] == "sigmoid":
  predictions = sigmoid(output_values) >= 0.5
elif parameters_out['activation'] == "relu":
  predictions = ReLU(output_values) >= 0.5
predictions

array([ True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True,  True, False,  True,  True,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False, False,  True,  True, False,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True, False,  True,  True,  True,
        True,  True,  True,  True, False,  True, False,  True,  True,
        True, False,  True,  True, False])

In [55]:
pred=[]
for i in predictions:
  #print(i)
  if str(i) == 'False':
    pred.append(0)
  else:
    pred.append(1)


In [56]:
y_true = y_test
y_pred = pred

## Accuracy

In [57]:
#Accuracy
import numpy as np
def accuracy_score(y_true, y_pred):
        acc = np.sum(np.equal(y_true, y_pred)) / len(y_true)
        return acc
accuracy_score(y_true,y_pred)

0.5

## F1 score

In [58]:
y_list = df['Outcome'].tolist()
y_train = y_list[:700]
y_test = y_list[700:]

In [59]:
def F1_score(pred,original): #pass predicted and original labels to this function
    #matrix=np.zeros((2,2)) # form an empty matric of 2x2
    tp,tn,fp,fn= 0,0,0,0
    for i in range(0,len(pred)): #the confusion matrix is for 2 classes: 1,0
        #1=positive, 0=negative
        if int(pred[i])==1 and int(original[i])==1: 
            tp=tp+1
        elif int(pred[i])== 1 and int(original[i])==0:
            fp=fp+1
        elif int(pred[i])==0 and int(original[i])==1:
            fn=fn+1
        elif int(pred[i])==0 and int(original[i])==0:
            tn=tn+1
    precision=tp/(tp+fn)
    #print("Precision:",precision)
    recall=tp/(tp+fp)
    #print("Recall:",recall)
    f1=2*(precision*recall)/(precision+recall)
    print("F1 score:")

    #the above code adds up the frequencies of the tps,tns,fps,fns and a matrix is formed
    return f1

F1_score(y_pred,y_test)

F1 score:


0.5853658536585366

## L1 Regularization

In [60]:
l1_penality = 50
lam = 0.01
def optimizeL1(x, y,learning_rate,iterations,parameters, activation): 
    size = x.shape[0]
    weight = parameters["weight"] 
    bias = parameters["bias"]
    for i in range(iterations): 
      if activation == "sigmoid":
        sigma = sigmoid(np.dot(x, weight) + bias)
      elif activation == "relu":
        sigma = ReLU(np.dot(x, weight) + bias)
        

      #L1 
    
      X=x
      m = X.shape[0]
      w=weight
      X=X
      Y=y
      b=bias
      #print(m)
      #Prediction
      final_result = sigmoid(np.dot(w,X.T)+b)
      #print(X.shape)
      Y_T = np.array(Y).T
      #print(final_result.shape)
      cost = (-1/m)*(np.sum((Y_T*np.log(final_result)) + ((1-Y_T)*(np.log(1-final_result)))))  + (lam * (np.sum(w)))

      #Gradient calculation
      dw = (1/m)*(np.dot(X.T, (final_result-np.array(Y).T).T)) + lam
      db = (1/m)*(np.sum(final_result-np.array(Y).T))
      
      grads = {"dw": dw, "db": db}
      dw = grads["dw"]
      db = grads["db"]

      weight -= learning_rate * dw
      bias -= learning_rate * db 
    
    parameters["weight"] = weight
    parameters["bias"] = bias
    parameters["activation"] = activation
    return parameters

In [61]:
def trainl1(x, y, learning_rate,iterations,activation):
    parameters_out = optimizeL1(x, y, learning_rate, iterations ,init_parameters, activation)
    return parameters_out

In [73]:
#parameters_out = train(x_train, y_train, learning_rate = 0.001, iterations = 200, activation = 'sigmoid')
init_parameters = {} 
init_parameters["weight"] = np.random.normal( size=x.shape[1]) #np.zeros(x.shape[1])
init_parameters["bias"] = 0

import numpy
parameters_out_l1 = trainl1(numpy.array(x_train), numpy.array(y_train), learning_rate = 0.001, iterations = 200, activation = 'sigmoid')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: RuntimeWarning: invalid value encountered in multiply


In [74]:
parameters_out_l1

{'activation': 'sigmoid',
 'bias': -0.007102791293346055,
 'weight': array([ 0.80343308, -0.8636775 ,  0.0259146 , -2.79137118,  1.21686059,
        -1.36155568])}

In [80]:
#Predictions
output_values = np.dot(x_test, parameters_out_l1['weight'])+parameters_out_l1['bias']
if parameters_out_l1['activation'] == "sigmoid":
  predictions_l1 = sigmoid(output_values) >= 0.5
elif parameters_out_l1['activation'] == "relu":
  predictions_l1 = ReLU(output_values) >= 0.5
#predictions

pred_l1=[]
for i in predictions_l1:
  #print(i)
  if str(i) == 'False':
    pred_l1.append(0)
  else:
    pred_l1.append(1)

#Accuracy
import numpy as np
def accuracy_score(y_true, y_pred):
        acc = np.sum(np.equal(y_true, y_pred)) / len(y_true)
        return acc
accuracy_score(y_true,pred_l1)


0.6764705882352942

## L2 Regularization

In [65]:
l2_penality = 40
def optimizeL2(x, y,learning_rate,iterations,parameters, activation): 
    size = x.shape[0]
    weight = parameters["weight"] 
    bias = parameters["bias"]
    for i in range(iterations): 
      if activation == "sigmoid":
        sigma = sigmoid(np.dot(x, weight) + bias)
      elif activation == "relu":
        sigma = ReLU(np.dot(x, weight) + bias)
        

      #L2
      X=x
      Y=y
      m = X.shape[0]
      w=weight
      b=bias
    
    #Prediction
      final_result = sigmoid(np.dot(w,X.T)+b)
      Y_T = Y.T
      cost = (-1/m)*(np.sum((Y_T*np.log(final_result)) + ((1-Y_T)*(np.log(1-final_result)))))  + (lam * (np.sum(np.square(w))))

    #Gradient calculation
      dw = (1/m)*(np.dot(X.T, (final_result-Y.T).T)) + lam * w
      db = (1/m)*(np.sum(final_result-Y.T))
    
      grads = {"dw": dw, "db": db}
      dw = grads["dw"]
      db = grads["db"]

      weight -= learning_rate * dw
      bias -= learning_rate * db 
    
    parameters["weight"] = weight
    parameters["bias"] = bias
    parameters["activation"] = activation
    return parameters

In [66]:
def trainl2(x, y, learning_rate,iterations,activation):
    parameters_out_l2 = optimizeL2(x, y, learning_rate, iterations ,init_parameters, activation)
    return parameters_out_l2

In [78]:
init_parameters = {} 
init_parameters["weight"] = np.random.normal(size=x.shape[1]) #np.zeros(x.shape[1])
init_parameters["bias"] = 0

import numpy
parameters_out_l2 = trainl2(numpy.array(x_train), numpy.array(y_train), learning_rate = 0.001, iterations = 200, activation = 'relu')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in multiply


In [68]:
parameters_out_l2

{'activation': 'relu',
 'bias': -0.017239969034310566,
 'weight': array([ 0.06676587, -0.02407664, -0.00934541, -0.438724  ,  0.03362963,
         0.35033535])}

In [81]:
#Predictions
output_values = np.dot(x_test, parameters_out_l2['weight'])+parameters_out_l2['bias']
if parameters_out_l2['activation'] == "sigmoid":
  predictions_l2 = sigmoid(output_values) >= 0.5
elif parameters_out_l2['activation'] == "relu":
  predictions_l2 = ReLU(output_values) >= 0.5
#predictions

pred_l2=[]
for i in predictions_l2:
  #print(i)
  if str(i) == 'False':
    pred_l2.append(0)
  else:
    pred_l2.append(1)

accuracy_score(y_true,pred_l2)


0.45588235294117646